# Reproducibilidad de experimentos

In [1]:
import sys
import os
import torch
import random

import torchaudio.transforms    as T
import torch.optim              as optim
import pandas                   as pd
import numpy                    as np

from torchvision       import transforms
from torch.utils.data  import DataLoader
from torch             import nn
from sklearn.metrics   import balanced_accuracy_score, recall_score, precision_score, mean_absolute_error, average_precision_score

sys.path.append(os.path.abspath(os.path.join('..','05-Train-Test')))
from utilit_train_test import make_weights_for_balanced_classes

sys.path.append(os.path.abspath(os.path.join('..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTimev2, permute_spec, smoothing_label
from Model             import iESPnet
from TrainEval         import train_model_v2, test_model_v2, get_performance_indices
from IO                import get_spectrogram_2

sys.path.append(os.path.abspath(os.path.join('../../..','03 Dynamic-Spatial-Filtering')))
from models            import DynamicSpatialFilter

torch.set_printoptions(precision=10)

# set the seed for reproducibility
torch.manual_seed(0)
random.seed(0)


In [2]:
# direccion donde se encuentran los espectrogramas 
SPE_DIR        = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_EEG/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)

In [3]:
# Variables iESPnet
FREQ_MASK_PARAM    = 10
TIME_MASK_PARAN    = 20
N_CLASSES          = 1
learning_rate      = 1e-3
batch_size         = 64    #128
epochs             = 20
num_workers        = 4

save_path          = 'models_DSF_iESPnet_prueba/'
patients           = df_meta['rns_id'].unique().tolist()

In [4]:
# Variables DSF
denoising          = 'autoreject'   # 'autoreject' 'data_augm' 
model              = 'stager_net'
dsf_type           = 'dsfd'         # 'dsfd' 'dsfm_st'
mlp_input          = 'log_diag_cov'
dsf_soft_thresh    = False
dsf_n_out_channels = None
n_channels         = 4

In [5]:
# hiperparametros iESPnet
hparams = {
           "n_cnn_layers" : 3,
           "n_rnn_layers" : 3,
           "rnn_dim"      : [150, 100, 50],
           "n_class"      : N_CLASSES,
           "out_ch"       : [8,8,16],
           "dropout"      : 0.3,
           "learning_rate": learning_rate,
           "batch_size"   : batch_size,
           "num_workers"  : num_workers,
           "epochs"       : epochs
          }

In [6]:
# define train y test de df_meta
test_id  = ['PIT-RNS1090', 'PIT-RNS8973', 'PIT-RNS1438', 'PIT-RNS8326', 'PIT-RNS3016']
vali_id  = ['PIT-RNS1603', 'PIT-RNS1556', 'PIT-RNS1534', 'PIT-RNS6989', 'PIT-RNS2543', 'PIT-RNS7168', 'PIT-RNS6762']

In [7]:
train_df = df_meta.copy() # hace falta resetear el indice de train_df?
test_df  = pd.DataFrame()
vali_df  = pd.DataFrame()

In [8]:
for s in range (len(test_id)):
    test_df = pd.concat([test_df, df_meta[df_meta['rns_id'] == test_id[s]]])
    test_df.reset_index(drop=True, inplace=True)
    train_df.drop(train_df[train_df['rns_id'] == test_id[s]].index, inplace = True)

for s in range(len(vali_id)):
    vali_df=pd.concat([vali_df, df_meta[df_meta['rns_id'] == vali_id[s]]])
    vali_df.reset_index(drop=True, inplace=True)
    train_df.drop(train_df[train_df['rns_id'] == vali_id[s]].index, inplace = True)

In [9]:
# experimentos que se van a realizar
experiments_1 = ['exp1','exp2','exp3']
experiments_2 = ['.1','.2','.3']

In [10]:
s = 0
j = 0

In [11]:
model1 = DynamicSpatialFilter(
                              n_channels, 
                              mlp_input            = mlp_input, 
                              n_out_channels       = dsf_n_out_channels, 
                              apply_soft_thresh    = dsf_soft_thresh
                             )

In [12]:
model2 = iESPnet(
                 hparams['n_cnn_layers'],
                 hparams['n_rnn_layers'],
                 hparams['rnn_dim'],
                 hparams['n_class'],
                 hparams['out_ch'],
                 hparams['dropout'],
                )

In [13]:
save_runs        = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/runs/'
save_models      = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/models/'
save_predictions = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/results/'
save_figs        = save_path + experiments_1[s] + '/' + str(experiments_1[s]) + str(experiments_2[j]) + '/figs/'

if not os.path.exists(save_path):
    os.makedirs(save_path)

if not os.path.exists(save_runs):
    os.makedirs(save_runs)

if not os.path.exists(save_models):
    os.makedirs(save_models)

if not os.path.exists(save_predictions):
    os.makedirs(save_predictions)

if not os.path.exists(save_figs):
    os.makedirs(save_figs)

In [14]:
print('Running training for: ' + experiments_1[s] +  experiments_2[j])

Running training for: exp1.1


In [15]:
# Dataloaders creados
train_data = SeizureDatasetLabelTimev2(
                                       file             = train_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None, 
                                       target_transform = smoothing_label(),
                                      )

In [16]:
# testing data should be balanced, just be "as it is"
test_data  = SeizureDatasetLabelTimev2(
                                       file             = test_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None,
                                       target_transform = smoothing_label()  
                                      )

In [17]:
# validation data should be balanced, just be "as it is"
vali_data  = SeizureDatasetLabelTimev2(
                                       file             = vali_df,
                                       root_dir         = SPE_DIR,
                                       transform        = None,
                                       target_transform = smoothing_label()  
                                      )

In [18]:
# data augmentation 
transform_train = transforms.Compose([
                                      T.FrequencyMasking(FREQ_MASK_PARAM),
                                      T.TimeMasking(TIME_MASK_PARAN), 
                                      permute_spec()                                                                     
                                    ])

In [19]:
weights = make_weights_for_balanced_classes(train_df, [0,1], n_concat=1)
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights, len(weights))

In [20]:
outputfile = save_models + 'model_' + str(experiments_1[s] + experiments_2[j])

In [21]:
'''

avg_train_losses, train_accs, avg_valid_losses, valid_accs = train_model_v2(
                                                                            model1, 
                                                                            model2, 
                                                                            hparams, 
                                                                            epochs, 
                                                                            train_data, 
                                                                            vali_data, 
                                                                            transform_train, 
                                                                            sampler, 
                                                                            outputfile,
                                                                            experiments_1[s],
                                                                            experiments_2[j]
                                                                           )

'''                                                                          

'\n\navg_train_losses, train_accs, avg_valid_losses, valid_accs = train_model_v2(\n                                                                            model1, \n                                                                            model2, \n                                                                            hparams, \n                                                                            epochs, \n                                                                            train_data, \n                                                                            vali_data, \n                                                                            transform_train, \n                                                                            sampler, \n                                                                            outputfile,\n                                                                            experiments_1[s],\n                           

In [22]:
avg_train_losses = []
train_accs       = []

In [23]:
avg_valid_losses = [] 
valid_accs       = []

In [24]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print('Using {} device'.format(device))

Using cuda device


In [25]:
# following pytorch suggestion to speed up training
torch.backends.cudnn.benchmark = False

In [26]:
kwargs = {'num_workers': hparams["num_workers"], 'pin_memory': True} if use_cuda else {}

train_loader = DataLoader(train_data, batch_size = hparams["batch_size"], sampler = sampler, **kwargs)
valid_loader = DataLoader(vali_data, batch_size = hparams["batch_size"], shuffle = False, **kwargs)

In [27]:
#move model1 to device
model1.to(device)

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [28]:
#move model2 to device
model2.to(device)

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [29]:
print('Num Model Parameters', sum([param1.nelement() for param1 in model1.parameters()]))
print('Num Model Parameters', sum([param2.nelement() for param2 in model2.parameters()]))

Num Model Parameters 120
Num Model Parameters 1654837


In [30]:
optimizer1 = optim.AdamW(model1.parameters(), hparams['learning_rate'], weight_decay=1e-4)
optimizer2 = optim.AdamW(model2.parameters(), hparams['learning_rate'], weight_decay=1e-4)

scheduler1 = optim.lr_scheduler.OneCycleLR(
                                           optimizer1, 
                                           max_lr          = hparams['learning_rate'], 
                                           steps_per_epoch = int(len(train_loader)),
                                           epochs          = hparams['epochs'],
                                           anneal_strategy = 'linear'
                                          )

scheduler2 = optim.lr_scheduler.OneCycleLR(
                                           optimizer2, 
                                           max_lr          = hparams['learning_rate'], 
                                           steps_per_epoch = int(len(train_loader)*2),
                                           epochs          = hparams['epochs'],
                                           anneal_strategy = 'linear'
                                          )
      
criterion = nn.BCEWithLogitsLoss().to(device)

In [31]:
epoch = 1

In [32]:
'''

train_losses, train_aucpr = training_DSF_iESPnet(
                                                 model1, 
                                                 model2, 
                                                 device, 
                                                 train_loader, 
                                                 transform_train, 
                                                 criterion, 
                                                 optimizer1, 
                                                 optimizer2, 
                                                 scheduler1, 
                                                 scheduler2, 
                                                 epoch,
                                                 experiment_1,
                                                 experiment_2
                                                )

'''                                                

'\n\ntrain_losses, train_aucpr = training_DSF_iESPnet(\n                                                 model1, \n                                                 model2, \n                                                 device, \n                                                 train_loader, \n                                                 transform_train, \n                                                 criterion, \n                                                 optimizer1, \n                                                 optimizer2, \n                                                 scheduler1, \n                                                 scheduler2, \n                                                 epoch,\n                                                 experiment_1,\n                                                 experiment_2\n                                                )\n\n'

In [33]:
experiment_1 = experiments_1[s]
experiment_2 = experiments_2[j]

In [34]:
# create spectrogram
ECOG_SAMPLE_RATE = 250
ECOG_CHANNELS    = 4
TT               = 1000 # window length
SPEC_WIN_LEN     = int(ECOG_SAMPLE_RATE * TT / 1000 ) # win size
overlap          = 500 
SPEC_HOP_LEN     = int(ECOG_SAMPLE_RATE * (TT - overlap) / 1000) # Length of hop between windows.
SPEC_NFFT        = 500  # to see changes in 0.5 reso
if   experiment_2 == '.1':  
    top_db       = 40.0
elif experiment_2 == '.2':
    top_db       = 60.0
elif experiment_2 == '.3':
    top_db       = 80.0

In [35]:
train_loss   = 0.0
train_losses = []
cont         = 0

In [36]:
model1.train()

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [37]:
model2.train()

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [38]:
for batch_idx, _data in enumerate(train_loader):
    cont+=1
    eeg, labels = _data 
    eeg, labels = eeg.to(device), labels.to(device)
    break
    

In [39]:
# Zero the gradients
optimizer1.zero_grad(set_to_none=True)
optimizer2.zero_grad(set_to_none=True)

utilizando la semilla random.seed el data loader siempre decide por el mismo batchsize

In [40]:
eeg[0,0,:]

tensor([-0.3365942538,  0.4565646350,  0.3017098010,  ...,
        -0.2752188444, -0.4093004763, -0.4093004763], device='cuda:0')

In [41]:
# Perform forward pass to DSF
outputs1 = model1(eeg)  # (batch, n_class)
outputs1 = outputs1.squeeze(1)

In [42]:
outputs1[0,0,:]

tensor([1.0571230650, 1.5284662247, 1.5069520473,  ..., 1.4071314335,
        1.3824717999, 1.4296525717], device='cuda:0', grad_fn=<SliceBackward0>)

In [43]:
if   experiment_1 == 'exp1': # sin normalizacion 
    pass
elif experiment_1 == 'exp2': # normalizacion por canal
    mean     = outputs1.mean(dim=2, keepdim=True)
    std      = outputs1.std(dim=2, keepdim=True)

    outputs1 = (outputs1 - mean) / std

elif experiment_1 == 'exp3': # normalizacion global
    outputs1 = (outputs1 - outputs1.mean()) / outputs1.std()

In [44]:
outputs1 = outputs1.to('cpu')

In [45]:
outputs1[0,0,:]

tensor([1.0571230650, 1.5284662247, 1.5069520473,  ..., 1.4071314335,
        1.3824717999, 1.4296525717], grad_fn=<SliceBackward0>)

In [46]:
# create spectrogram from outputs1
spectrograms = get_spectrogram_2(outputs1, device, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)

In [47]:
spectrograms[0,0,:,:]

array([[42.1847  , 40.824608, 41.287724, ..., 40.95363 , 41.19576 ,
        40.61157 ],
       [41.368523, 39.98082 , 40.294914, ..., 39.981346, 40.271187,
        39.644917],
       [38.8527  , 37.600483, 37.191147, ..., 37.011093, 37.43944 ,
        36.699203],
       ...,
       [20.063866, 20.063866, 20.063866, ..., 20.063866, 20.063866,
        20.063866],
       [20.063866, 20.063866, 20.063866, ..., 20.063866, 20.063866,
        20.063866],
       [20.063866, 20.063866, 20.063866, ..., 20.063866, 20.063866,
        20.063866]], dtype=float32)

In [48]:
spectrograms = torch.from_numpy(spectrograms)

In [49]:
spectrograms[0,0,:,:]

tensor([[42.1847000122, 40.8246078491, 41.2877235413,  ...,
         40.9536285400, 41.1957588196, 40.6115684509],
        [41.3685226440, 39.9808197021, 40.2949142456,  ...,
         39.9813461304, 40.2711868286, 39.6449165344],
        [38.8526992798, 37.6004829407, 37.1911468506,  ...,
         37.0110931396, 37.4394416809, 36.6992034912],
        ...,
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616]])

In [50]:
spectrograms_transformed = transform_train(spectrograms)

In [51]:
spectrograms_transformed[0,0,:,:]

tensor([[36.9560470581, 38.5045242310, 37.5006332397,  ...,
         37.6816177368, 38.4132537842, 36.1781272888],
        [35.9786453247, 37.6109352112, 36.4316062927,  ...,
         36.6866188049, 37.5390396118, 35.0619659424],
        [32.9985466003, 34.9101295471, 33.0317802429,  ...,
         33.5982856750, 34.9096374512, 31.5700321198],
        ...,
        [ 0.0000000000,  0.0000000000,  0.0000000000,  ...,
          0.0000000000,  0.0000000000,  0.0000000000],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616]])

In [52]:
spectrograms2train = torch.cat((spectrograms, spectrograms_transformed), axis=0)

In [53]:
spectrograms2train[0,0,:,:]

tensor([[42.1847000122, 40.8246078491, 41.2877235413,  ...,
         40.9536285400, 41.1957588196, 40.6115684509],
        [41.3685226440, 39.9808197021, 40.2949142456,  ...,
         39.9813461304, 40.2711868286, 39.6449165344],
        [38.8526992798, 37.6004829407, 37.1911468506,  ...,
         37.0110931396, 37.4394416809, 36.6992034912],
        ...,
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616]])

In [54]:
spectrograms2train = spectrograms2train.to(device)

In [55]:
spectrograms2train[0,0,:,:]

tensor([[42.1847000122, 40.8246078491, 41.2877235413,  ...,
         40.9536285400, 41.1957588196, 40.6115684509],
        [41.3685226440, 39.9808197021, 40.2949142456,  ...,
         39.9813461304, 40.2711868286, 39.6449165344],
        [38.8526992798, 37.6004829407, 37.1911468506,  ...,
         37.0110931396, 37.4394416809, 36.6992034912],
        ...,
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616],
        [20.0638656616, 20.0638656616, 20.0638656616,  ...,
         20.0638656616, 20.0638656616, 20.0638656616]], device='cuda:0')

In [56]:
labels2train = torch.cat((labels, labels), axis=0)

In [57]:
outputs2 = model2(spectrograms2train)

In [58]:
outputs2[0,:]

tensor([-0.1099266931, -0.0395331606,  0.0182540305,  0.0263280943,
        -0.0239245072, -0.0094657391, -0.0076280385,  0.0017163977,
         0.0278404504,  0.0821563751, -0.0951414481,  0.0532986745,
         0.1602554023, -0.0038349330, -0.0478094742, -0.0330767632,
        -0.0897817090, -0.0956881419, -0.1592417955,  0.1090546548,
        -0.2096019089, -0.0089410096,  0.0682112724, -0.0225163996,
         0.1059725285, -0.0218000188, -0.1951575279, -0.0371089503,
        -0.0463627949,  0.0276790634, -0.0093379505, -0.1291644275,
         0.0505096503,  0.0106447712, -0.1384189725, -0.0052026324,
         0.0304810740,  0.0873709843,  0.0260692015, -0.0704263374,
        -0.0805853978,  0.0205334313, -0.1467180848,  0.0488160327,
        -0.0355344638,  0.1028430089, -0.0578082651,  0.1216111854,
         0.0409928113,  0.0202666745,  0.0417032465, -0.0568032935,
        -0.0639105365, -0.0860885754, -0.0076957941, -0.1232442930,
        -0.0745031163, -0.0032451749, -0.0013194

In [59]:
m     = nn.Sigmoid()
probs = m(outputs2)

In [60]:
probs[0,:]

tensor([0.4725459814, 0.4901179671, 0.5045633316, 0.5065816641, 0.4940191507,
        0.4976335764, 0.4980929792, 0.5004290938, 0.5069596767, 0.5205275416,
        0.4762325585, 0.5133215189, 0.5399783254, 0.4990412593, 0.4880499244,
        0.4917315543, 0.4775696397, 0.4760962427, 0.4602735043, 0.5272366405,
        0.4477905333, 0.4977647364, 0.5170462132, 0.4943711162, 0.5264683962,
        0.4945501983, 0.4513648748, 0.4907238483, 0.4884113669, 0.5069193244,
        0.4976654947, 0.4677537084, 0.5126247406, 0.5026611686, 0.4654504061,
        0.4986993670, 0.5076196790, 0.5218288898, 0.5065169334, 0.4824006557,
        0.4798645377, 0.5051331520, 0.4633860886, 0.5122016072, 0.4911173284,
        0.5256881118, 0.4855519533, 0.5303654075, 0.5102467537, 0.5050665140,
        0.5104243159, 0.4858029783, 0.4840277731, 0.4784911275, 0.4980759919,
        0.4692278504, 0.4813828170, 0.4991886616, 0.4996701181, 0.5042982101,
        0.5062740445, 0.5068084598, 0.4864051342, 0.5121046901, 

In [61]:
y_true  = torch.max(labels2train, dim = 1)[0]
y_pred  = torch.max(probs, dim = 1)[0]

In [62]:
if cont == 1:
    Y_true = y_true
    Y_pred = y_pred

else:                
    Y_true = torch.cat((Y_true, y_true), axis=0)
    Y_pred = torch.cat((Y_pred, y_pred), axis=0)

In [63]:
y_true

tensor([1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
        1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0.,
        1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0., 1.,
        1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0.,
        1., 1.], device='cuda:0', dtype=torch.float64)

In [64]:
y_pred 

tensor([0.5579710603, 0.5462026596, 0.5523999929, 0.5542988777, 0.5525311232,
        0.5610816479, 0.5778387785, 0.5527662635, 0.5605447888, 0.5505529642,
        0.5458220243, 0.5618414879, 0.5538496375, 0.5609320998, 0.5619035959,
        0.5702580810, 0.5705679655, 0.5629058480, 0.5540416837, 0.5693856478,
        0.5790207386, 0.5796253085, 0.5572024584, 0.5537558198, 0.5563972592,
        0.5757946968, 0.5582665205, 0.5702161193, 0.5476136804, 0.5592188835,
        0.5790522099, 0.5578448176, 0.5698016286, 0.5597276688, 0.5554230809,
        0.5547875166, 0.5514724851, 0.5515486598, 0.5522392392, 0.5644313693,
        0.5666114688, 0.5795406103, 0.5634781122, 0.5712374449, 0.5551255941,
        0.5729779601, 0.5706380606, 0.5635226369, 0.5622778535, 0.5587263107,
        0.5673288703, 0.5635853410, 0.5617873669, 0.5587468743, 0.5516604185,
        0.5812342763, 0.5577102304, 0.5524936914, 0.5495192409, 0.5840418935,
        0.5641591549, 0.5678297281, 0.5519568324, 0.5747315288, 

In [65]:
loss = criterion(outputs2, labels2train)

In [66]:
loss

tensor(0.6936606664, device='cuda:0', dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [ ]:
loss.backward()
train_loss += loss.item()

In [ ]:
# Perform optimization
optimizer1.step()
optimizer2.step()
scheduler1.step()
scheduler2.step()

In [ ]:
# record training loss
train_losses.append(loss.item())

In [ ]:
del _data
torch.cuda.empty_cache()

In [ ]:
y_val_true, val_pred = Y_true.to('cpu').detach().numpy(), Y_pred.to('cpu').detach().numpy()

In [ ]:
y_val_true

array([1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1.,
       0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1.,
       1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 1., 1.])

In [ ]:
val_pred 

array([0.55796456, 0.54619944, 0.5524026 , 0.5543061 , 0.5525338 ,
       0.56106955, 0.57785237, 0.5527633 , 0.5605436 , 0.5505509 ,
       0.5458222 , 0.56183714, 0.5538384 , 0.5609299 , 0.5618995 ,
       0.5702769 , 0.5705745 , 0.5629146 , 0.55404925, 0.56939214,
       0.57901895, 0.57962173, 0.55720323, 0.5537612 , 0.55639607,
       0.57579565, 0.55826926, 0.5702225 , 0.54760915, 0.55922836,
       0.5790541 , 0.5578461 , 0.56979257, 0.5597291 , 0.55542076,
       0.5547884 , 0.5514792 , 0.55154955, 0.5522426 , 0.56443703,
       0.5666142 , 0.5795472 , 0.56347203, 0.57123345, 0.5551218 ,
       0.5729913 , 0.5706498 , 0.56353045, 0.562281  , 0.5587333 ,
       0.5673164 , 0.5635918 , 0.5617815 , 0.5587699 , 0.5516663 ,
       0.5812272 , 0.5577105 , 0.5524921 , 0.54951537, 0.58403665,
       0.56416506, 0.5678289 , 0.5519585 , 0.5747406 , 0.57100874,
       0.55639464, 0.5566398 , 0.5959412 , 0.56610674, 0.55510867,
       0.578736  , 0.58056575, 0.572433  , 0.55046815, 0.55015

In [ ]:
np.mean(train_losses)

np.float64(0.693661426627358)

In [ ]:
train_aucpr = average_precision_score(y_val_true,val_pred)

In [ ]:
train_aucpr

np.float64(0.7191555105653817)

In [ ]:
'''

valid_losses, valid_aucpr = validate_v2(
                                        model1,
                                        model2,
                                        device, 
                                        valid_loader, 
                                        criterion, 
                                        epoch,
                                        experiment_1,
                                        experiment_2
                                       )

'''

'\n\nvalid_losses, valid_aucpr = validate_v2(\n                                        model1,\n                                        model2,\n                                        device, \n                                        valid_loader, \n                                        criterion, \n                                        epoch,\n                                        experiment_1,\n                                        experiment_2\n                                       )\n\n'

In [ ]:
ECOG_SAMPLE_RATE = 250
ECOG_CHANNELS    = 4
TT               = 1000 
SPEC_WIN_LEN     = int(ECOG_SAMPLE_RATE * TT / 1000 ) 
overlap          = 500 
SPEC_HOP_LEN     = int(ECOG_SAMPLE_RATE * (TT - overlap) / 1000) 
SPEC_NFFT        = 500
if   experiment_2 == '.1':  
    top_db       = 40.0
elif experiment_2 == '.2':
    top_db       = 60.0
elif experiment_2 == '.3':
    top_db       = 80.0

In [ ]:
valid_losses = []
cont         = 0

In [ ]:
model1.eval()

DynamicSpatialFilter(
  (feat_extractor): SpatialFeatureExtractor()
  (mlp): Sequential(
    (0): Linear(in_features=4, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=20, bias=True)
  )
)

In [ ]:
model2.eval()

iESPnet(
  (freqcnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(120, 1), stride=(1, 1), padding=(119, 0), dilation=(2, 1), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (timecnn): Sequential(
    (0): Conv2d(4, 8, kernel_size=(1, 181), stride=(1, 1), padding=(0, 180), dilation=(1, 2), bias=False)
    (1): ReLU()
    (2): InstanceNorm2d(8, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
  )
  (cnn_ori): Conv2d(4, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (cnn): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 1), padding=(1, 1), bias=False)
  (rescnn_layers): Sequential(
    (0): ResidualCNNbatch(
      (cnn1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (cnn2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (dropout1): Dropout(p=0.3, inplace=False)
      (dropout2): Dropout(p=0.3, inpla

In [ ]:
with torch.no_grad():
    for eeg, labels in valid_loader:
        cont+=1
    
        eeg, labels = eeg.to(device), labels.to(device)
        break

In [ ]:
eeg[0,0,:]

tensor([-0.4312779605,  0.7666466832,  0.7666466832,  ...,
         1.7282834053,  2.3272457123,  2.3272457123], device='cuda:0')

In [ ]:
outputs1    = model1(eeg)
outputs1    = outputs1.squeeze(1)

In [ ]:
outputs1[0,0,:]

tensor([ 1.2036125660, -0.5157686472, -0.2858802676,  ...,
        -0.3922327757, -1.0043334961, -0.0676615238], device='cuda:0',
       grad_fn=<SliceBackward0>)

In [ ]:
if   experiment_1 == 'exp1': # sin normalizacion 
    pass
elif experiment_1 == 'exp2': # normalizacion por canal
    mean     = outputs1.mean(dim=2, keepdim=True)
    std      = outputs1.std(dim=2, keepdim=True)

    outputs1 = (outputs1 - mean) / std

elif experiment_1 == 'exp3': # normalizacion global
    outputs1 = (outputs1 - outputs1.mean()) / outputs1.std()

In [ ]:
outputs1 = outputs1.to('cpu')

In [ ]:
outputs1[0,0,:]

tensor([ 1.2036125660, -0.5157686472, -0.2858802676,  ...,
        -0.3922327757, -1.0043334961, -0.0676615238], grad_fn=<SliceBackward0>)

In [ ]:
spectrograms = get_spectrogram_2(outputs1, device, ECOG_SAMPLE_RATE, SPEC_NFFT, SPEC_WIN_LEN, SPEC_HOP_LEN, top_db)

In [ ]:
spectrograms[0,0,:,:]

array([[36.805565, 34.840134, 34.31321 , ..., 34.417633, 34.76141 ,
        30.557869],
       [36.02135 , 33.790146, 33.390934, ..., 33.623505, 33.93907 ,
        29.05682 ],
       [33.74846 , 30.42043 , 30.663383, ..., 31.240124, 31.486313,
        23.876953],
       ...,
       [ 3.133751,  9.328862, 11.79624 , ..., 11.992373, 10.561933,
        10.242782],
       [ 3.133751,  9.636835, 12.209043, ...,  9.976776, 11.249703,
        11.043509],
       [ 3.133751, 10.14909 , 11.124491, ...,  9.342586, 12.069448,
        11.25875 ]], dtype=float32)

In [ ]:
spectrograms = torch.from_numpy(spectrograms)
spectrograms = spectrograms.to(device)
spectrograms[0,0,:,:]

tensor([[36.8055648804, 34.8401336670, 34.3132095337,  ...,
         34.4176330566, 34.7614097595, 30.5578689575],
        [36.0213508606, 33.7901458740, 33.3909339905,  ...,
         33.6235046387, 33.9390716553, 29.0568199158],
        [33.7484588623, 30.4204292297, 30.6633834839,  ...,
         31.2401237488, 31.4863128662, 23.8769531250],
        ...,
        [ 3.1337509155,  9.3288621902, 11.7962398529,  ...,
         11.9923734665, 10.5619325638, 10.2427816391],
        [ 3.1337509155,  9.6368350983, 12.2090425491,  ...,
          9.9767761230, 11.2497034073, 11.0435085297],
        [ 3.1337509155, 10.1490898132, 11.1244907379,  ...,
          9.3425855637, 12.0694484711, 11.2587499619]], device='cuda:0')

In [ ]:
outputs2 = model2(spectrograms)

In [ ]:
outputs2[0,:]

tensor([ 0.0268676206, -0.0102258921, -0.0349138603, -0.0334215462,
        -0.0234079435, -0.0166705623, -0.0309853628, -0.0297762901,
        -0.0336499438, -0.0154534802, -0.0290622711, -0.0222812146,
        -0.0254983902, -0.0360274985, -0.0286205560, -0.0244271457,
        -0.0182656869, -0.0261916965, -0.0296516716, -0.0395974293,
        -0.0171713606, -0.0233043209, -0.0273038149, -0.0303875357,
        -0.0247275308, -0.0386262238, -0.0436143354, -0.0479446277,
        -0.0290906653, -0.0391142368, -0.0387745947, -0.0319819450,
        -0.0314857587, -0.0314369947, -0.0393586233, -0.0308798328,
        -0.0312369391, -0.0353200138, -0.0257927105, -0.0194795951,
        -0.0309981108, -0.0301626176, -0.0393772423, -0.0464365110,
        -0.0310236886, -0.0146152452, -0.0253396258, -0.0161633492,
        -0.0305184573, -0.0249036178, -0.0248050913, -0.0222530439,
        -0.0183672905, -0.0357629880, -0.0341870040, -0.0478546768,
        -0.0350622833, -0.0398877934, -0.0384599

In [ ]:
m = nn.Sigmoid()
probs = m(outputs2)

In [ ]:
probs[0,:]

tensor([0.5031316280, 0.4923630059, 0.4882842898, 0.4877800941, 0.4902320802,
        0.4924897850, 0.4887770116, 0.4896684289, 0.4886046052, 0.4933273792,
        0.4897944629, 0.4911900163, 0.4900868535, 0.4885035157, 0.4901591241,
        0.4897157848, 0.4910209179, 0.4901307821, 0.4896396399, 0.4864921272,
        0.4920317531, 0.4907867610, 0.4892153442, 0.4889356196, 0.4904960692,
        0.4874242246, 0.4849896431, 0.4852839112, 0.4888647497, 0.4869835377,
        0.4862327576, 0.4887111783, 0.4885486960, 0.4882102311, 0.4870402515,
        0.4898419380, 0.4896703959, 0.4881352484, 0.4904320538, 0.4915081561,
        0.4885614514, 0.4893611968, 0.4873071611, 0.4853157997, 0.4888099730,
        0.4920020998, 0.4902067482, 0.4931480289, 0.4894672930, 0.4906793535,
        0.4902188778, 0.4919020236, 0.4925732911, 0.4874093831, 0.4880323112,
        0.4850503504, 0.4882877767, 0.4876057804, 0.4880566299, 0.4871735871,
        0.4867578745, 0.4908548295, 0.4923704565, 0.4892777205, 

In [ ]:
if len(probs.shape) == 1:
    probs.unsqueeze_(0)
    outputs2.unsqueeze_(0)

y_true  = torch.max(labels, dim =1)[0]
y_pred  = torch.max(probs, dim=1)[0]

if cont == 1:
    Y_true = y_true
    Y_pred= y_pred

else:                
    Y_true = torch.cat((Y_true, y_true), axis=0)
    Y_pred = torch.cat((Y_pred, y_pred), axis=0)

In [ ]:
loss = criterion(outputs2, labels)
valid_losses.append(loss.item())

In [ ]:
loss

tensor(0.6687840343, device='cuda:0', dtype=torch.float64,
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [ ]:
del eeg, loss
torch.cuda.empty_cache()    

In [ ]:
valid_aucpr = average_precision_score(Y_true.to('cpu').detach().numpy(), Y_pred.to('cpu').detach().numpy())

/home/martin/Documentos/environments/env_thalamus/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:993: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(


In [ ]:
np.mean(valid_losses), valid_aucpr

(np.float64(0.6687840342847116), np.float64(-0.0))